In [7]:
# Importing libraries and API endpoints

import pandas as pd

from sklearn.preprocessing import LabelEncoder
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

from datetime import timedelta
from datetime import datetime

from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import boxscoreadvancedv3
from nba_api.stats.endpoints import boxscorescoringv2
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.static import teams
from nba_api.stats.endpoints import boxscorescoringv3
from nba_api.live.nba.endpoints import playbyplay
from sbrscrape import Scoreboard

# Extracting raw game data from NBA API

In [6]:
# Testing endpoint
# pd.DataFrame(teams.get_teams())
# boxscore = boxscoreadvancedv3.BoxScoreAdvancedV3().team_stats.get_data_frame()
# boxscore_df = pd.DataFrame(boxscore)
# boxscore_df.columns

help(boxscoreadvancedv2)

Help on module nba_api.stats.endpoints.boxscoreadvancedv2 in nba_api.stats.endpoints:

NAME
    nba_api.stats.endpoints.boxscoreadvancedv2

CLASSES
    nba_api.stats.endpoints._base.Endpoint(builtins.object)
        BoxScoreAdvancedV2
    
    class BoxScoreAdvancedV2(nba_api.stats.endpoints._base.Endpoint)
     |  BoxScoreAdvancedV2(game_id, end_period='0', end_range='0', range_type='0', start_period='0', start_range='0', proxy=None, headers=None, timeout=30, get_request=True)
     |  
     |  Method resolution order:
     |      BoxScoreAdvancedV2
     |      nba_api.stats.endpoints._base.Endpoint
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(self, game_id, end_period='0', end_range='0', range_type='0', start_period='0', start_range='0', proxy=None, headers=None, timeout=30, get_request=True)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  get_request(self)
     |  
     |  load_response(self)


In [11]:
gamelogs_df = teamgamelogs.TeamGameLogs(season_nullable='2023-24').team_game_logs.get_data_frame()

In [12]:
# Sort sequentially by Game ID
gamelogs_df = gamelogs_df.sort_values(by='GAME_ID', ascending=True)

In [106]:
# Initial preprocessing
# Dropping all columns with suffix "_RANK" and other irrelevant columns (PLUS_MINUS is the point diff which we FE)
gamelogs_df = gamelogs_df.drop(columns=gamelogs_df.filter(regex='_RANK').columns)
gamelogs_df = gamelogs_df.drop(columns=['AVAILABLE_FLAG', 'PLUS_MINUS'])

In [274]:
# efg
# fta rate
# to ratio (TO * 100) / (FGA + (FTA * 0.44) + AST + TO
# oreb%
# dreb%
# trb%
# ast%
# stl%
# blk%
# tov%
# ortg
# drtg
# previous season winning pct

gamelogs_df['EFG%'] = ((gamelogs_df['FGM'] + (0.5 * gamelogs_df['FG3M'])) / gamelogs_df['FGA'])
gamelogs_df['FTA RATE'] = (gamelogs_df['FTA'] / gamelogs_df['FGA'])
gamelogs_df['TS%'] = gamelogs_df['PTS'] / (2 * (gamelogs_df['FGA'] + 0.44 * gamelogs_df['FTA']))
gamelogs_df['AST%'] = gamelogs_df['AST'] / gamelogs_df.groupby('GAME_ID')['AST'].transform('sum')
gamelogs_df['STL%'] = gamelogs_df['STL'] / gamelogs_df.groupby('GAME_ID')['STL'].transform('sum')
gamelogs_df['BLK%'] = gamelogs_df['BLK'] / gamelogs_df.groupby('GAME_ID')['BLK'].transform('sum')
gamelogs_df['TOV%'] = gamelogs_df['TOV'] / gamelogs_df.groupby('GAME_ID')['TOV'].transform('sum')
gamelogs_df['TOV RATIO'] = (gamelogs_df['TOV'] * 100) / gamelogs_df['POSS']

In [ ]:
gamelogs_df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,PTS,EFG%,FTA RATE,OREB%,DREB%,TRB%,AST%,STL%,BLK%,TOV%
1135,2023-24,1610612739,CLE,Cleveland Cavaliers,0022300001,2023-11-03T00:00:00,CLE @ IND,L,48.0,44,...,116,0.571429,0.297619,0.416667,0.476190,0.466667,0.490909,0.588235,0.642857,0.406250
1131,2023-24,1610612754,IND,Indiana Pacers,0022300001,2023-11-03T00:00:00,IND vs. CLE,W,48.0,45,...,121,0.610465,0.279070,0.583333,0.523810,0.533333,0.509091,0.411765,0.357143,0.593750
1126,2023-24,1610612749,MIL,Milwaukee Bucks,0022300002,2023-11-03T00:00:00,MIL vs. NYK,W,48.0,35,...,110,0.548780,0.341463,0.304348,0.459459,0.422680,0.571429,0.500000,0.916667,0.560000
1132,2023-24,1610612752,NYK,New York Knicks,0022300002,2023-11-03T00:00:00,NYK @ MIL,L,48.0,38,...,105,0.447917,0.260417,0.695652,0.540541,0.577320,0.428571,0.500000,0.083333,0.440000
1128,2023-24,1610612764,WAS,Washington Wizards,0022300003,2023-11-03T00:00:00,WAS @ MIA,L,48.0,46,...,114,0.648148,0.209877,0.363636,0.464286,0.447761,0.485294,0.437500,0.800000,0.487805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,2023-24,1610612756,PHX,Phoenix Suns,0022301228,2023-12-08T00:00:00,PHX vs. SAC,L,48.0,40,...,106,0.548193,0.277108,0.347826,0.477612,0.444444,0.509804,0.533333,1.000000,0.458333
667,2023-24,1610612754,IND,Indiana Pacers,0022301229,2023-12-07T00:00:00,IND @ MIL,W,48.0,50,...,128,0.524510,0.245098,0.555556,0.514286,0.525773,0.612245,0.750000,0.470588,0.454545
669,2023-24,1610612749,MIL,Milwaukee Bucks,0022301229,2023-12-07T00:00:00,MIL vs. IND,L,48.0,43,...,119,0.510417,0.291667,0.444444,0.485714,0.474227,0.387755,0.250000,0.529412,0.545455
666,2023-24,1610612740,NOP,New Orleans Pelicans,0022301230,2023-12-07T00:00:00,NOP @ LAL,L,48.0,34,...,89,0.394737,0.210526,0.521739,0.384615,0.415842,0.415094,0.538462,0.555556,0.444444


In [47]:
from requests.exceptions import ReadTimeout

# def retry_with_timeout(func):
#     @wraps(func)
#     def wrapper(*args, **kwargs):
#         max_attempts = kwargs.pop('max_attempts', 3)
#         delay = kwargs.pop('delay', 0.6)  # Sleep time between retries
#         attempts = 0
#         while attempts < max_attempts:
#             try:
#                 return func(*args, **kwargs)
#             except ReadTimeout as e:
#                 print(f"Timeout error occurred: {e}")
#                 attempts += 1
#                 time.sleep(delay)  # Sleep before retrying
#         raise ReadTimeout(f"Function {func.__name__} failed after {max_attempts} attempts")

#     return wrapper

boxscore = []

unique_game_ids = gamelogs_df['GAME_ID'].unique()
unique_game_ids_100 = unique_game_ids[:100]

for id in unique_game_ids_100:
    # try:
        boxscore_df = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id).team_stats.get_data_frame()
        boxscore.append(boxscore_df)
    # except ReadTimeout as e:
    #     print(f"Timeout error occurred for game ID: {id}. Moving to the next game ID.")

if boxscore:
    boxscore_df = pd.concat(boxscore, ignore_index=True)

boxscore_df

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
boxscore_cols = ['gameId', 'teamId', 'estimatedOffensiveRating', 'offensiveRating',
       'estimatedDefensiveRating', 'defensiveRating', 'estimatedNetRating',
       'netRating', 'assistPercentage', 'assistToTurnover', 'assistRatio',
       'offensiveReboundPercentage', 'defensiveReboundPercentage',
       'reboundPercentage', 'estimatedTeamTurnoverPercentage', 'turnoverRatio',
       'effectiveFieldGoalPercentage', 'trueShootingPercentage',
       'estimatedPace', 'pace',
       'pacePer40', 'possessions', 'PIE']

boxscore_df = boxscore_df[boxscore_cols]

In [51]:
boxscore_df.rename(columns={'gameId': 'GAME_ID', 'teamId': 'TEAM_ID'}, inplace=True)

In [52]:
gamelogs_df = pd.merge(gamelogs_df, boxscore_df, on=['GAME_ID', 'TEAM_ID'])

In [53]:
gamelogs_df

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,2023-24,1610612741,CHI,Chicago Bulls,0022300004,2023-11-03T00:00:00,CHI vs. BKN,L,48.0,43,...,10.5,0.545,0.562,1.0,0.202,98.68,94.5,78.75,95.0,0.484
1,2023-24,1610612751,BKN,Brooklyn Nets,0022300004,2023-11-03T00:00:00,BKN @ CHI,W,48.0,44,...,11.7,0.552,0.555,1.0,0.194,98.68,94.5,78.75,94.0,0.516


In [327]:
ignore_columns = ['SEASON_YEAR', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
                  'GAME_DATE', 'MATCHUP', 'WL', 'MIN']
columns_to_diff = gamelogs_df.drop(columns=ignore_columns).columns

# Convert relevant columns to numeric
gamelogs_df[columns_to_diff] = gamelogs_df[columns_to_diff].apply(pd.to_numeric, errors='coerce')

# Calculate the difference for the specified columns grouped by 'GAME_ID'. The first row will be nan. The second row will have the diff.
diff_df = gamelogs_df.groupby('GAME_ID')[columns_to_diff].diff()
# .rename(columns=lambda x: f'{x}_DIFF')

# Identify all duplicates in gamelogs_df['GAME_ID']
first_row_mask = gamelogs_df.duplicated(subset=['GAME_ID'])

# The first instance of each game_id is equal to the negative of the second row.
diff_df.loc[~first_row_mask] =  -diff_df.loc[first_row_mask].values

In [328]:
# Drop the redundant columns and concat the diff_df to the final gamelogs_diff_df
gamelogs_diff_df = gamelogs_df.drop(columns=columns_to_diff)
gamelogs_diff_df = pd.concat([gamelogs_diff_df, diff_df], axis=1)

In [329]:
columns_to_drop = ['TEAM_NAME', 'TEAM_ABBREVIATION', 'GAME_DATE', 'MIN', 'GAME_ID', 'MATCHUP']
gamelogs_diff_df = gamelogs_diff_df.drop(columns=columns_to_drop)
gamelogs_diff_df['WL'] = LabelEncoder().fit_transform(gamelogs_diff_df['WL'])

In [330]:
gamelogs_diff_df

,SEASON_YEAR,TEAM_ID,WL,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,...,EFG%,FTA RATE,TOV RATIO,OREB%,DREB%,TRB%,AST%,STL%,BLK%,TOV%
1135,2023-24,1610612739,0,-1.0,-2.0,0.001,-7.0,-3.0,-0.198,4.0,...,-0.039037,0.018549,-3.605255,-0.166667,-0.047619,-0.066667,-0.018182,0.176471,0.285714,-0.187500
1131,2023-24,1610612754,1,1.0,2.0,-0.001,7.0,3.0,0.198,-4.0,...,0.039037,-0.018549,3.605255,0.166667,0.047619,0.066667,0.018182,-0.176471,-0.285714,0.187500
1126,2023-24,1610612749,1,-3.0,-14.0,0.031,10.0,-0.0,0.257,1.0,...,0.100864,0.081047,2.492176,-0.391304,-0.081081,-0.154639,0.142857,-0.000000,0.833333,0.120000
1132,2023-24,1610612752,0,3.0,14.0,-0.031,-10.0,0.0,-0.257,-1.0,...,-0.100864,-0.081047,-2.492176,0.391304,0.081081,0.154639,-0.142857,0.000000,-0.833333,-0.120000
1128,2023-24,1610612764,0,-2.0,1.0,-0.032,-0.0,1.0,-0.017,-3.0,...,-0.033102,0.034877,-0.635814,-0.272727,-0.071429,-0.104478,-0.029412,-0.125000,0.600000,-0.024390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,2023-24,1610612756,0,-2.0,-9.0,0.025,-3.0,-7.0,-0.015,-1.0,...,0.015584,0.059717,-0.912259,-0.304348,-0.044776,-0.111111,0.019608,0.066667,1.000000,-0.083333
667,2023-24,1610612754,1,7.0,6.0,0.042,-5.0,-0.0,-0.152,-0.0,...,0.014093,-0.046569,-2.077317,0.111111,0.028571,0.051546,0.224490,0.500000,-0.058824,-0.090909
669,2023-24,1610612749,0,-7.0,-6.0,-0.042,5.0,0.0,0.152,0.0,...,-0.014093,0.046569,2.077317,-0.111111,-0.028571,-0.051546,-0.224490,-0.500000,0.058824,0.090909
666,2023-24,1610612740,0,-13.0,9.0,-0.189,-10.0,-4.0,-0.260,-8.0,...,-0.250612,-0.126683,-1.653706,0.043478,-0.230769,-0.168317,-0.169811,0.076923,0.111111,-0.111111


In [331]:
gamelogs_diff_df[diff_df.columns] = gamelogs_diff_df.groupby(['SEASON_YEAR', 'TEAM_ID'])[diff_df.columns].transform('mean')

In [332]:
gamelogs_diff_df = gamelogs_diff_df.drop(columns=['SEASON_YEAR', 'TEAM_ID'])
gamelogs_diff_df = gamelogs_diff_df.drop(columns=['FGM', 'PTS'])
# gamelogs_diff_df = gamelogs_diff_df.drop(columns='FGA')

In [333]:
X = gamelogs_diff_df.drop(columns='WL')
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

y = gamelogs_diff_df['WL']

clf_lr = LogisticRegression(random_state=42)
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)

sfs_lr = SequentialFeatureSelector(clf_lr, 
                                k_features='best',
                                forward=True,
                                floating=False,
                                scoring='accuracy',
                                cv=5
                                )
 
sfs_lr.fit(X_scaled, y)

sfs_rf = SequentialFeatureSelector(clf_rf, 
                                k_features='best',
                                forward=True,
                                floating=False,
                                scoring='accuracy',
                                cv=5
                                )
 
sfs_rf.fit(X, y)

# Selected features with Logistic Regression as algorithm
selected_feature_index_lr = sfs_lr.k_feature_idx_
selected_feature_names_lr = X.columns[list(selected_feature_index_lr)]

# Selected features with Random Forest as algorithm
selected_feature_index_rf = sfs_rf.k_feature_idx_
selected_feature_names_rf = X.columns[list(selected_feature_index_rf)]
print("Selected Features using Logistic Regression:", selected_feature_names_lr)
print("Selected Features using Random Forest:", selected_feature_names_rf)

/Users/kelvin/opt/anaconda3/envs/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kelvin/opt/anaconda3/envs/python/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

Selected Features using Logistic Regression: Index(['FGA', 'FG3A', 'PF', 'EFG%', 'TRB%', 'STL%'], dtype='object')
Selected Features using Random Forest: Index(['FTM'], dtype='object')


# Extracting raw betting data using SBRScrape

In [220]:
# Initialize dataframes
df = pd.DataFrame()
bets_df = pd.DataFrame()

In [11]:
# Testing endpoint
Scoreboard(sport="NBA", date='2023-11-08').games


[{'date': '2023-11-09T00:00:00+00:00',
  'status': 'Final',
  'home_team': 'Indiana Pacers',
  'home_team_loc': 'Indiana',
  'home_team_abbr': 'IND',
  'home_team_rank': -1,
  'away_team': 'Utah Jazz',
  'away_team_loc': 'Utah',
  'away_team_abbr': 'UTA',
  'away_team_rank': -1,
  'home_score': 134,
  'away_score': 118,
  'home_spread': {'betvictor': -6.5,
   'tonybet': -6.5,
   'bet365': -6,
   'pinnacle': -6,
   'sportsinteraction': -5.5,
   '888sport': -6.5,
   'comeon': -6,
   'caesars': -6.5,
   'betmgm': -5.5,
   'bet_rivers_ny': -6,
   'pointsbet': -6,
   'betway': -5.5,
   'fanduel': -6,
   'draftkings': -5.5},
  'home_spread_odds': {'betvictor': -105,
   'tonybet': -104,
   'bet365': -110,
   'pinnacle': -104,
   'sportsinteraction': -115,
   '888sport': -110,
   'comeon': -114,
   'caesars': -110,
   'betmgm': -115,
   'bet_rivers_ny': -110,
   'pointsbet': -110,
   'betway': -120,
   'fanduel': -112,
   'draftkings': -115},
  'away_spread': {'betvictor': 6.5,
   'tonybet': 6

In [250]:
import pandas as pd
from datetime import datetime, timedelta
from sbrscrape import Scoreboard  # Assuming Scoreboard is imported from the sbrscrape package

# Set the start date to October 1, 2023
start_date = datetime(2023, 10, 25)

# Create an empty list to store DataFrames for each date
dfs = []

# Initialize the current date to the start date
current_date = start_date

# Set the end date to December 31, 2023
end_date = datetime(2023, 11, 10)

# Fetch data for each date until the end date is reached
while current_date <= end_date:
    try:
        # the date parameter operates weirdly, seems to get the day after the input date?
        games = Scoreboard(sport="NBA", date=current_date).games
        u_df = pd.DataFrame(games)
        dfs.append(u_df)
        print(f"Data fetched for {current_date}")
    except Exception as e:
        print(f"Error fetching data for {current_date}: {e}")

    current_date += timedelta(days=1)

# Concatenate intermediate df into the final bets_df
if dfs:
    df = pd.concat(dfs, ignore_index=True)
    bets_df = pd.DataFrame(df)
    print("Concatenation successful.")
else:
    print("No data fetched. Check for errors during data retrieval.")


Data fetched for 2023-10-25 00:00:00
Data fetched for 2023-10-26 00:00:00
Data fetched for 2023-10-27 00:00:00
Data fetched for 2023-10-28 00:00:00
Data fetched for 2023-10-29 00:00:00
Data fetched for 2023-10-30 00:00:00
Data fetched for 2023-10-31 00:00:00
Data fetched for 2023-11-01 00:00:00
Data fetched for 2023-11-02 00:00:00
Data fetched for 2023-11-03 00:00:00
Data fetched for 2023-11-04 00:00:00
Data fetched for 2023-11-05 00:00:00
Data fetched for 2023-11-06 00:00:00
Error fetching data for 2023-11-07 00:00:00: 'Scoreboard' object has no attribute 'games'
Data fetched for 2023-11-08 00:00:00
Data fetched for 2023-11-09 00:00:00
Data fetched for 2023-11-10 00:00:00
Concatenation successful.


In [263]:
pd.DataFrame(dfs)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

In [265]:
pd.DataFrame(dfs)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

In [13]:
home_spread_odds_df = bets_df['home_spread_odds'].apply(pd.Series).add_suffix('_home_spread_odds')
away_spread_odds_df = bets_df['away_spread_odds'].apply(pd.Series).add_suffix('_away_spread_odds')
under_odds_df = bets_df['under_odds'].apply(pd.Series).add_suffix('_under_odds')
over_odds_df = bets_df['over_odds'].apply(pd.Series).add_suffix('_over_odds')
home_ml_df = bets_df['home_ml'].apply(pd.Series).add_suffix('_home_ml')
away_ml_df = bets_df['away_ml'].apply(pd.Series).add_suffix('_away_ml')

df_games = pd.concat([bets_df, home_ml_df, away_ml_df], axis=1)

columns_to_drop = ['home_spread', 'home_spread_odds', 'away_spread', 'away_spread_odds', 'under_odds', 'over_odds', 'total', 'home_ml', 'away_ml']
bets_df = bets_df.drop(columns_to_drop, axis=1)

In [14]:
moneyline_columns = [
    'betvictor_away_ml', 'tonybet_away_ml', 'bet365_away_ml', 'pinnacle_away_ml', 
    'sportsinteraction_away_ml', '888sport_away_ml', 'comeon_away_ml', 'caesars_away_ml', 
    'betmgm_away_ml', 'bet_rivers_ny_away_ml', 'pointsbet_away_ml', 'betway_away_ml', 
    'fanduel_away_ml', 'draftkings_away_ml',
    'betvictor_home_ml', 'tonybet_home_ml', 'bet365_home_ml', 'pinnacle_home_ml', 
    'sportsinteraction_home_ml', '888sport_home_ml', 'comeon_home_ml', 'caesars_home_ml', 
    'betmgm_home_ml', 'bet_rivers_ny_home_ml', 'pointsbet_home_ml', 'betway_home_ml', 
    'fanduel_home_ml', 'draftkings_home_ml'
]

for column in moneyline_columns:
    odds = []

    for odd in df_games[column]:
        if odd < 0:
            ip = (-1 * odd) / ((-1 * odd) + 100)
        else:
            ip = 100 / (odd + 100)
        
        odds.append(ip)

    bets_df[column + '_iprob'] = odds

In [15]:
columns_to_drop = ['home_team_loc', 'away_team_loc', 'home_team_abbr', 'away_team_abbr', 'home_team_rank', 'away_team_rank']
# df_games.drop(columns=moneyline_columns, inplace=True)
bets_df.drop(columns=columns_to_drop, inplace=True)

In [16]:
bets_df

,date,status,home_team,away_team,home_score,away_score,betvictor_away_ml_iprob,tonybet_away_ml_iprob,bet365_away_ml_iprob,pinnacle_away_ml_iprob,...,sportsinteraction_home_ml_iprob,888sport_home_ml_iprob,comeon_home_ml_iprob,caesars_home_ml_iprob,betmgm_home_ml_iprob,bet_rivers_ny_home_ml_iprob,pointsbet_home_ml_iprob,betway_home_ml_iprob,fanduel_home_ml_iprob,draftkings_home_ml_iprob
0,2023-10-25T23:00:00+00:00,Final,Indiana Pacers,Washington Wizards,143,120,0.392157,0.395257,0.370370,0.377358,...,0.666667,0.666667,0.671053,0.763593,0.666667,0.661017,0.655172,0.655172,0.672131,0.666667
1,2023-10-25T23:00:00+00:00,Final,Charlotte Hornets,Atlanta Hawks,116,110,0.625468,0.641577,0.629630,0.635036,...,0.400000,0.425532,0.392157,0.442478,0.400000,0.411523,0.408163,0.416667,0.396825,0.378788
2,2023-10-25T23:00:00+00:00,Final,Orlando Magic,Houston Rockets,116,86,0.408163,0.406504,0.377358,0.389105,...,0.649123,0.652778,0.632353,0.616858,0.649123,0.638989,0.655172,0.636364,0.647887,0.677419
3,2023-10-25T23:00:00+00:00,Final,New York Knicks,Boston Celtics,104,108,0.625468,0.636364,0.629630,0.624060,...,0.408163,0.416667,0.408163,0.458716,0.408163,0.411523,0.408163,0.425532,0.406504,0.434783
4,2023-10-25T23:30:00+00:00,Final,Miami Heat,Detroit Pistons,103,102,0.232558,0.230415,0.222222,0.224719,...,0.818182,0.833333,0.847561,0.806576,0.818182,0.827586,0.809524,0.818182,0.818182,0.826087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,2024-01-01T00:00:00+00:00,Final,San Antonio Spurs,Boston Celtics,101,134,0.909091,0.925926,0.909091,0.917695,...,0.121212,0.125000,0.054054,0.125000,0.121212,0.117647,0.111111,0.142857,0.117647,0.117647
475,2024-01-01T00:00:00+00:00,Final,New Orleans Pelicans,Los Angeles Lakers,129,109,0.333333,0.357143,0.357143,0.354610,...,0.692308,0.689441,0.999001,0.689441,0.692308,0.714286,0.692308,0.692308,0.677419,0.685535
476,2024-01-01T00:00:00+00:00,Final,Oklahoma City Thunder,Brooklyn Nets,124,108,0.312500,0.298507,0.289855,0.294985,...,0.750000,0.769053,0.999001,0.746193,0.750000,0.753086,0.750000,0.722222,0.743590,0.748744
477,2024-01-01T01:00:00+00:00,Final,Phoenix Suns,Orlando Magic,112,107,0.350877,0.357143,0.350877,0.333333,...,0.692308,0.714286,0.970871,0.694190,0.692308,0.705882,0.692308,0.692308,0.692308,0.696970
